# **1. Install and Import Dependencies**

In [33]:
!pip3 install transformers
!pip3 install datasets
!pip3 install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.1 MB/s eta 0:00:00


In [45]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import torch
import os
import numpy as np
import evaluate
from google.colab import drive
import pandas as pd
from datasets import load_dataset

# **2. Load dataset**

In [22]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
os.chdir('/content/drive/MyDrive/Semesters/2023 - SPRING/CPS 5801 - Advanced Artificial Intelligence/AI Term Project')

In [53]:
dataset = load_dataset("csv", data_files='IMDB Dataset.csv', split=['train[:80%]', 'train[80%:]'])
dataset_train = dataset[0]
dataset_test = dataset[1]

  0%|          | 0/2 [00:00<?, ?it/s]

In [54]:
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 40000
})
Dataset({
    features: ['review', 'sentiment'],
    num_rows: 10000
})


In [56]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

def tokenize_function(sample):
  return tokenizer(sample["review"], padding="max_length", truncation=True)


tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_test = dataset_test.map(tokenize_function, batched=True)

print(tokenized_train)
print(tokenized_test)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 40000
})
Dataset({
    features: ['review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})


DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [57]:
train_dataset = tokenized_train.shuffle(seed=80)
eval_dataset = tokenized_test.shuffle(seed=80)

# **3. Fine-tune pre-trained model**

In [30]:
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment", num_labels=2, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
training_args = TrainingArguments(output_dir='test_trainer', evaluation_strategy="epoch")

In [35]:
metric = evaluate.load("accuracy")

In [37]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [58]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_test,
    compute_metrics=compute_metrics
)

In [59]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: ignored


# **3. Encode and Calculate Sentiment**

In [ ]:
tokens = tokenizer.encode("I disliked this movie, but it was better than the second film.", return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
result.logits

tensor([[-1.4703,  0.4868,  1.7897,  0.6267, -1.3403]],
       grad_fn=<AddmmBackward0>)

In [ ]:
int(torch.argmax(result.logits))+1

3

# **4. Collect Reviews**

# **5. Load Reviews into DataFrame and Score**

In [ ]:
print('Review | Classification | Score')
for index, row in df.iterrows():
  tokens = tokenizer.encode(row['review'], truncation=True, padding=True, return_tensors= 'pt')
  result = model(tokens)
  score = int(torch.argmax(result.logits)) + 1
  print(row['review'][:20], row['sentiment'], score)

Review | Classification | Score
One of the other rev positive 3
A wonderful little p positive 5
I thought this was a positive 4
Basically there's a  negative 3
Petter Mattei's "Lov positive 4
Probably my all-time positive 5
I sure would like to positive 3
This show was an ama negative 2
Encouraged by the po negative 1
If you like original positive 5
Phil the Alien is on negative 3
I saw this movie whe negative 2
So im not a big fan  negative 3
The cast played Shak negative 2
This a fantastic mov positive 5
Kind of drawn in by  negative 1
Some films just simp positive 3
This movie made it i negative 1


KeyboardInterrupt: ignored